In [ ]:
from utz import *
from IPython.display import HTML, SVG, display

import fiona
import geopandas as gpd
from geopy import distance
import shapefile
from shapely.geometry import Point
from shapely.ops import cascaded_union, unary_union

import plotly.express as px
import plotly.graph_objects as go

import sys
sys.executable

## Mapbox configs

In [ ]:
token = open(".mapbox-token").read()
mapbox = dict(mapbox=dict(
    style="dark",
    accesstoken=token,
))

## Load county shapes

In [ ]:
fiona.listlayers("tlgdb_2022_a_34_nj.gdb/")

In [ ]:
fiona.listlayers("tlgdb_2022_a_34_nj_edges.gdb/")

In [ ]:
counties = gpd.read_file("tlgdb_2022_a_34_nj.gdb/", layer='County')
counties

In [ ]:
cgeos = counties.geometry.tolist()
cgeos

In [ ]:
[ len(c.boundary.geoms) for c in cgeos ]

In [ ]:
counties_union = unary_union(counties.geometry.tolist())
counties_union

In [ ]:
type(counties_union)

In [ ]:
type(counties_union.boundary)

In [ ]:
nj_boundary = counties_union.boundary
nj_boundary

In [ ]:
nj_bnd_lines = list(nj_boundary.geoms)
nj_bnd_lines

In [ ]:
[ display(ln) for ln in nj_bnd_lines ]

In [ ]:
ln = nj_bnd_lines[0]
ln

In [ ]:
ln.length

In [ ]:
[ ln.length for ln in nj_bnd_lines ]

In [ ]:
list(nj_boundary.geoms)

In [ ]:
len(counties)

In [ ]:
def is_nj_ll(lat, lon):
    point = Point(lon, lat)
    return counties_union.contains(point)

def is_nj(r):
    return is_nj_ll(r.LAT, r.LON)

In [ ]:
def get_county(r):
    hits = counties[counties.geometry.contains(Point(r.LON, r.LAT))].NAMELSAD
    if len(hits) > 1:
        err(f"{r}: {len(hits)} counties: {hits}")
        return
    elif hits.empty:
        return
    [county] = hits.tolist()
    return county

## County / State Boundaries

In [ ]:
bounds = counties.geometry.boundary.to_list()
bounds

In [ ]:
[len(b.geoms[0].coords) for b in bounds]

In [ ]:
counties.geometry.boundary.tolist()

In [ ]:
def county_points(r):
    [name] = r.index.unique()
    [multilinestr] = r.geometry.boundary.tolist()
    linestrs = multilinestr.geoms
    if len(linestrs) > 1:
        err(f'{name}: {len(linestrs)} linestrings')
    return Series([ c for l in linestrs for c in l.coords ], name='point')

In [ ]:
county_coords = (
    counties
    .rename(columns={'NAMELSAD': 'name'})
    .groupby('name')
    .apply(county_points)
    .reset_index(level=1, drop=True)
    .reset_index()
)
county_coords

In [ ]:
county_coords.point.value_counts().value_counts().sort_index()

In [ ]:
%%time
bnd_lls = sxs(county_coords.name, county_coords.point.apply(lambda p: Series(p, index=['lon', 'lat'])))
bnd_lls

In [ ]:
ll_hist = bnd_lls[['lat','lon']].value_counts()
ll1s = ll_hist[ll_hist == 1]
ll_hist

In [ ]:
ll1s.index.to_series()

In [ ]:
p1s = bnd_lls.merge(ll1s, left_on=['lat', 'lon'], right_index=True).drop(columns='count')
p1s

In [ ]:
fig = px.scatter_mapbox(
    p1s,
    lat="lat", lon="lon",
    color='name',
    #hover_name="City",
    hover_data=["name"],
    #color_discrete_sequence=["yellow", "orange", "red"],
    center=dict(lat=40.15, lon=-74.715),
    zoom=7.6,
    height=1000,
)
legend_bgcolor = '50'
fig.update_layout(
    **mapbox,
    title=dict(
        text="NJ boundary points, by county",
        x=0.5, y=0.98,
        xanchor='center', yanchor='top',
        font=dict(size=32, color="white")
    ),
    legend=dict(
        title=dict(text=''),
        x=0.98, y=0.98,
        xanchor="right", yanchor="top",
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    ),
    margin={"r":0,"t":0,"l":0,"b":0},
)
fig.write_image('nj-boundary-by-county.png', width=1000, height=1000)
fig.show()